## Teste Técnico - Programa Trainee triggo.ai 2025

### Other packages used in this notebook

In [ ]:
# %pip install numpy
# %pip install pandas

# %pip install duckdb
# %pip install kagglehub

# %pip install matplotlib
# %pip install seaborn

# %pip install geopy

### Imports

In [ ]:
import numpy as np
import pandas as pd
import duckdb
import kagglehub

import matplotlib.pyplot as plt
import seaborn as sns

from geopy import distance 


from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay

# Part 1:

## Data Loading and Cleaning

This project is divided in 4 parts and each part have it's own particular questions:
- Data cleaning and modelling.
    - Importing the dataset .csv file from Kaggle ("olistbr/brazilian-ecommerce")
    - Data cleaning and normalizing
    - Creation of a relational data modelling
    - Describing each step in the data prep

### Kaggle dataset download

In [ ]:
# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

### Reading the datasets into Pandas DataFrames

Listing all the 9 datasets and describing them:

- olist_customers_dataset.csv
    - Contains information about the customers
    - Raw column names: customer_id, customer_unique_id, customer_zip_code_prefix, customer_city and customer_state
- olist_geolocation_dataset.csv
    - Contains geolocalization information of every geolocation in the dataset
    - Raw column names: geolocation_zip_code_prefix, geolocation_lat, geolocation_lng, geolocation_city, geolocation_state
- olist_order_items_dataset.csv
    - Contains information about the customers
    - Raw column names: order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value
- olist_order_payments_dataset.csv
    - Contains information the orders payments
    - Raw column names: order_id, payment_sequential, payment_type, payment_installments, payment_value
- olist_order_reviews_dataset.csv
    - Contains information about the reviews from customers for every order
    - Raw column names: review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timest
- olist_orders_dataset.csv
    - Contains information about orders details
    - Raw column names: order_id,customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date,order_estimated_delivery_date
- olist_products_dataset
    - Contains information about products details
    - Raw column names: product_id, product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
- olist_sellers_dataset
    - Contains information about the sellers
    - Raw column names: seller_id, seller_zip_code_prefix, seller_city, seller_state
- product_category_name_translation
    - Contains name translations, wont be used in this project

![Table Diagram](<Table Diagram.png>)

- Defining a function to convert timestamp to date
- Defining a function to fill null comments

In [ ]:
def convert_to_date(df, column_name):
    for col in column_name:
        df[col] = pd.to_datetime(df[col]).dt.date
    return df

def fill_null(df, column_name, value):
    for col in column_name:
        df[col] = df[col].fillna(value = value)
    return df

### Read the olist_customers_dataset.csv into df_customers


In [ ]:
dtype = {'customer_zip_code_prefix': 'string'}

df_customers = pd.read_csv(path + r"\olist_customers_dataset.csv", dtype=dtype)

df_customers['customer_city'] = df_customers['customer_city'].str.title()

df_customers.head()

### Read the olist_geolocation_dataset.csv into df_geolocation


In [ ]:
dtype = {'geolocation_zip_code_prefix': 'string'}

df_geolocation = pd.read_csv(path + r"\olist_geolocation_dataset.csv", dtype=dtype)

df_geolocation['geolocation_city'] = df_geolocation['geolocation_city'].str.title()

df_geolocation.head()

### Read the olist_order_items_dataset.csv into df_order_items


In [ ]:
parse_dates = ['shipping_limit_date']

df_order_items = pd.read_csv(path + r"\olist_order_items_dataset.csv", parse_dates=parse_dates)

df_order_items = convert_to_date(df_order_items, parse_dates)

df_order_items.head()

### Read the olist_order_payments_dataset.csv to df_order_payments


In [ ]:
df_order_payments = pd.read_csv(path + r"\olist_order_payments_dataset.csv")

df_order_payments['payment_type'] = df_order_payments['payment_type'].str.replace("_", " ")
df_order_payments['payment_type'] = df_order_payments['payment_type'].str.title()

df_order_payments.head()

### Read the olist_order_reviews_dataset.csv to df_order_reviews


In [ ]:
parse_dates = ['review_creation_date', 'review_answer_timestamp']
comment_columns = ['review_comment_title', 'review_comment_message']

df_order_reviews = pd.read_csv(path + r"\olist_order_reviews_dataset.csv", parse_dates=parse_dates)

df_order_reviews = convert_to_date(df_order_reviews, parse_dates)

df_order_reviews = fill_null(df_order_reviews, comment_columns, 'No review')

df_order_reviews.head()

### Read the olist_orders_dataset.csv to df_orders


In [ ]:
parse_dates = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

df_orders = pd.read_csv(path + r"\olist_orders_dataset.csv", parse_dates=parse_dates)

df_orders = convert_to_date(df_orders, parse_dates)

df_orders.head()

### Read the olist_products_dataset.csv to df_products

Remove underlines and capitalize first letters from product_category_name

In [ ]:
df_products = pd.read_csv(path + r"\olist_products_dataset.csv")

df_products['product_category_name'] = df_products['product_category_name'].str.replace("_", " ")
df_products['product_category_name'] = df_products['product_category_name'].str.title()

df_products = fill_null(df=df_products, column_name=['product_category_name'], value='No Category')

df_products.head()

### Read the olist_sellers_dataset.csv to df_sellers

- First letters up using the title() method

In [ ]:
dtype = {'seller_zip_code_prefix': 'string'}

df_sellers = pd.read_csv(path + r"\olist_sellers_dataset.csv", dtype=dtype)

df_sellers['seller_city'] = df_sellers['seller_city'].str.title()

df_sellers.head()

# Part 2:

## Exploratory Analysis

- Exploratory analysis using SQL and Python
    - Orders volume per month and its seasonality
    - The distribution of orders delivery time
    - The relationship between delivery costs and delivery distance
    - Which are the best-selling products in revenue
    - Brazilian states with the highest average order value

### Orders volume per month and its seasonality

Here I used datetrunc to get the months of every date, then format with strftime, to get only a Month date. If datepart was used it would return an Integer instead of a String, so I prefer this way.

In [ ]:
orders_per_month = duckdb.query(
                                    """
                                    SELECT 
                                        strftime(datetrunc('month', order_purchase_timestamp), '%m') AS month, 
                                        COUNT(order_id) AS count_orders 
                                    FROM df_orders 
                                    GROUP BY month 
                                    ORDER BY month
                                    """

                                ).df()

orders_per_month

### Months of August, May and July were the top 3 orders per months

Im using iloc to get the first and second columns

In [ ]:
month = orders_per_month.iloc[:, 0]
count_orders = orders_per_month.iloc[:, 1]

plt.bar(month, count_orders)
plt.title('Orders per Month')
plt.xlabel('Month')
plt.ylabel('Orders')
plt.show()


### The Distribution of orders delivery time

To calculate the delivery time, Im using the order_delivered_customer_date as delivery time and the order_approved_at when the timer to the order be shipped

Calculating the mode of the dataset, to get the most frequent day to delivery

Selection only orders with the order_status = delivered

In [ ]:
order_delivery_time = duckdb.query(
                                    """
                                    SELECT
                                        order_id, 
                                        datediff('day', order_approved_at, order_delivered_customer_date) AS day_to_delivery 
                                    FROM df_orders 
                                    WHERE order_status = 'delivered'
                                    """

                                ).df()

print('Mode: ', order_delivery_time.day_to_delivery.mode())

In [ ]:
sns.displot(order_delivery_time, x="day_to_delivery", binwidth=3)

### SQL JOINs to get the right data from the multiple tables

- Starting with Customer table joining Geolocation table, to get the latitute and longitude from each customer
- As latitude and Longitude coulh have multiple values for each zip code, I decided to use the AVG function to get an average of each location
- Then joining Customer table with Orders table to get the order_id

In [ ]:
customer_geo = duckdb.query(
"""
  SELECT
    c.customer_unique_id,
    c.customer_zip_code_prefix,
    c.customer_city,
    c.customer_state,
    o.order_id,
    o.order_status,
    COALESCE(o.order_approved_at, o.order_purchase_timestamp) AS order_approved_at,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    AVG(g.geolocation_lat) AS customer_lat,
    AVG(g.geolocation_lng) AS customer_lng
  FROM
    df_customers AS c
  INNER JOIN
    df_geolocation AS g
      ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
      AND c.customer_city = g.geolocation_city
      AND c.customer_state = g.geolocation_state
  INNER JOIN 
    df_orders AS o
      ON c.customer_id = o.customer_id
      AND order_status = 'delivered'
  GROUP BY
    1,2,3,4,5,6,7,8,9
"""
).df()

customer_geo.head()

Joining the customer_geo table with df_order_items, to get some metrics and to join with df_products, to get products metrics, for usage in the freight calculation 

In [ ]:

order_products = duckdb.query(
"""
  SELECT
    cg.customer_unique_id,
    cg.customer_zip_code_prefix,
    cg.customer_city,
    cg.customer_state,
    cg.customer_lat,
    cg.customer_lng,
    cg.order_id,
    cg.order_status,
    cg.order_approved_at,
    cg.order_delivered_customer_date,
    cg.order_estimated_delivery_date, 
    oi.seller_id,
    oi.product_id,
    oi.price,
    oi.freight_value,
    p.product_category_name
  FROM 
    customer_geo AS cg
  INNER JOIN   
    df_order_items AS oi
      ON cg.order_id = oi.order_id
  INNER JOIN   
    df_products AS p
      ON oi.product_id = p.product_id
"""
).df()

order_products

- Joining the seller_df and the geolocation_df to get the seller localization
- Again, using the AVG function to get an average of each zip_code

In [ ]:

seller_geo = duckdb.query(
"""
  SELECT
    s.seller_id,
    s.seller_zip_code_prefix,
    s.seller_city,
    s.seller_state,
    AVG(g.geolocation_lat) AS seller_lat,
    AVG(g.geolocation_lng) AS seller_lng
  FROM
    df_sellers AS s
  INNER JOIN
    df_geolocation AS g
      ON s.seller_zip_code_prefix = g.geolocation_zip_code_prefix
      AND s.seller_city = g.geolocation_city
      AND s.seller_state = g.geolocation_state
  GROUP BY
    1,2,3,4
"""
).df()

seller_geo.head()

- Joining the order_products with the seller_geo df to get the sellers data
- Joining order_products with the df_order_reviews, and get the review score
- Use the coalesce function to give a 0 to the review_score in case it was NULL (the customer didnt review)

In [ ]:
master_table = duckdb.query(
"""
  SELECT
    op.order_id,
    coalesce(r.review_score,0) AS review_score,
    order_status,
    order_approved_at,
    order_delivered_customer_date,
    order_estimated_delivery_date, 

    product_id,
    price,
    freight_value,
    product_category_name,

    customer_unique_id,
    customer_zip_code_prefix,
    customer_city,
    customer_state,
    customer_lat,
    customer_lng,

    s.seller_id,
    s.seller_zip_code_prefix,
    s.seller_city,
    s.seller_state,
    seller_lat,
    seller_lng
  FROM
    order_products AS op
  INNER JOIN
    seller_geo AS s
      ON op.seller_id = s.seller_id
  LEFT JOIN
    df_order_reviews AS r
      ON op.order_id = r.order_id
"""
).df()

master_table

### Using the lib GeoPY its possible to calculate the distance between two point using the latitude and longitude

In [ ]:
def calculate_distance(row):
    customer_location = (row['customer_lat'], row['customer_lng'])
    seller_location = (row['seller_lat'], row['seller_lng'])
    return distance.distance(customer_location, seller_location).km


master_table['delivery_distance'] = master_table.apply(calculate_distance, axis=1)

master_table

In [ ]:
master_table_csv = duckdb.query(
"""
  SELECT
    order_id,
    review_score,
    order_status,
    order_approved_at,
    order_delivered_customer_date,
    order_estimated_delivery_date, 

    product_id,
    price,
    freight_value,
    product_category_name,

    customer_unique_id,
    customer_zip_code_prefix,
    customer_state,

    seller_id,
    seller_zip_code_prefix,
    seller_state,
    ROUND(delivery_distance, 2)
  FROM
    master_table
"""
).df()

master_table_csv

### Calculating the relationship between freigth cost and distance

In [ ]:
cents_per_km = duckdb.query(
"""
    SELECT
        order_id,
        product_id,
        product_category_name,
        freight_value,
        CASE WHEN delivery_distance < 1 THEN 1 ELSE delivery_distance END AS delivery_distance,
        freight_value / (CASE WHEN delivery_distance < 1 THEN 1 ELSE delivery_distance END) AS price_per_km  
    FROM
        master_table
"""
).df()

cents_per_km

### Analysis of the Freight values

Grouped by Product Category with its minimum, maximum and the average

In [ ]:
freight_analysis = duckdb.query(
"""
    SELECT
        product_category_name,
        MIN(price_per_km) AS min_price_per_km,
        MAX(price_per_km) AS max_price_per_km,
        AVG(price_per_km) AS avg_price_per_km
    FROM
        cents_per_km
    GROUP BY
        1
"""
).df()

freight_analysis

### Best Selling Products Categories

In [ ]:
best_selling_products = duckdb.query(
"""
    SELECT
        product_category_name,
        COUNT(product_id) AS count_products,
        SUM(price) AS sum_price
    FROM
        master_table
    GROUP BY
        1
    ORDER BY 
        3 DESC
"""
).df()

best_selling_products

### Which Brazilian States have the highest average order price 

In [ ]:
highest_average_orders = duckdb.query(
"""
    SELECT
        customer_state,
        COUNT(product_id) AS count_products,
        AVG(price) AS avg_price
    FROM
        master_table
    GROUP BY
        1
    ORDER BY 
        3 DESC
"""
).df()

highest_average_orders

# Part 3:

## Solving Business Problems

- Solving Business Problems
    - Retention analysis of recurring customers (one or more orders in the analysis period) and which insights could we extract
    - Late delivery prediction analysis:
        - Define what a late delivery is.
        - Use relevant fields to create the features for the model
        - Divide train and test datasets
        - Implement the prediction analisys using Random Forest Classifier
        - Evaluate the model performance and explain the results
    - Customer Segmentation: 
        - Segment the customers in groups
        - Do a behavior analysis  of each group and suggest marketing strategies for each one.
    - Customer Satisfaction: 
        - Explore the relationship between customer review score and other aspects, like product category, delivery time, average order price.
        - Identify the factors that impact the most

### Retention Analysis

Data preparation:
- Step 1: define the number of orders for each customer
- Step 2: calculate the order delivery On Time or Late and other metrics for both customers
- Step 3: analyse the results:
    - Average review score, Average Ticket, Quantity of products by State

In [ ]:
recurrent_customer_id = duckdb.query(
"""
    WITH recurrent_customer_id AS (
        SELECT
            customer_unique_id,
            order_id,
            order_approved_at,
            SUM(price) AS sum_price,     
            ROW_NUMBER() OVER(PARTITION BY customer_unique_id ORDER BY order_approved_at) AS order_number    
        FROM
            master_table
        GROUP BY
            1,2,3
    )
    SELECT
        customer_unique_id        
    FROM
        recurrent_customer_id
    WHERE
        order_number > 1
"""
).df()

recurrent_customer_id

In [ ]:
customer_orders = duckdb.query(
"""
    SELECT
        mt.customer_unique_id,
        CASE
            WHEN rc.customer_unique_id IS NOT NULL THEN 'Recurrent Customer'
            ELSE 'Single Order Customer'  
        END AS customer_classification,
        customer_state,

        order_id,
        product_category_name,
        order_approved_at,
        order_estimated_delivery_date,
        order_delivered_customer_date,
        ROUND(delivery_distance, 2) AS delivery_distance,

        CASE
            WHEN date_diff('day', order_approved_at, order_estimated_delivery_date) -
                 date_diff('day', order_approved_at, order_delivered_customer_date) >= 0 THEN 'On Time Delivery'
            ELSE 'Late Delivery'
        END AS delivery_status,

        COALESCE(MAX(review_score),0) AS review_score,
        COUNT(product_id) AS qty_products,
        SUM(price) AS sum_price        
    FROM
        master_table AS mt
    LEFT JOIN
        recurrent_customer_id AS rc
        ON mt.customer_unique_id = rc.customer_unique_id
    GROUP BY
        1,2,3,4,5,6,7,8,9
    ORDER BY
        2
"""
).df()

customer_orders

- Calculate for each customer, how many times each delivery status, the quantity of orders and products, the total spend and the average review score 

In [ ]:
customer_agg = duckdb.query(
"""
    SELECT
        customer_unique_id,
        customer_classification,
        customer_state,
        product_category_name,
        
        SUM(CASE WHEN delivery_status = 'On Time Delivery' THEN 1 ELSE 0 END) AS on_time_delivery,
        SUM(CASE WHEN delivery_status = 'Late Delivery' THEN 1 ELSE 0 END) AS late_delivery,

        COUNT(order_id) AS qty_orders,
        SUM(qty_products) AS qty_products,
        SUM(sum_price) AS total_spend,
        AVG(delivery_distance) AS avg_delivery_distance,
        AVG(review_score) AS avg_review_score
    FROM
        customer_orders
    GROUP BY
        1,2,3,4
    ORDER BY
        1
"""
).df()

customer_agg

- Agregating per state and calculating the delivery percent, quantity orders and products, total spend and the average delivery distance and review score

In [ ]:
customer_agg_recurrent = duckdb.query(
"""
    SELECT
        customer_state,
        
        ROUND(SUM(late_delivery) / SUM(qty_orders), 2) AS late_delivery_percent,

        SUM(qty_orders) AS qty_orders,
        SUM(qty_products) AS qty_products,
        ROUND(SUM(total_spend), 2) AS total_spend,
        ROUND(AVG(avg_delivery_distance), 2) AS avg_delivery_distance,
        ROUND(AVG(avg_review_score), 2) AS avg_review_score

    FROM
        customer_agg
    WHERE
        customer_classification = 'Recurrent Customer'
    GROUP BY
        1
    ORDER BY
        6 DESC
"""
).df()

customer_agg_recurrent

With this Analysis we can afirm:
- When the average distance of delivery is higher then 1400km, there's a huge drop in total sales
- The most distant State also have the highest Late Delivery Percentage and lowest Average Review Score
- SP is the biggest state in quantity of orders, products and spend and also have the lowest Average Delivery Distance
- The higher Average Review Score, the lowest Late Delivery Percentage is 

## Late delivery prediction analysis:
- Define what a late delivery is.
- Use relevant fields to create the features for the model
- Divide train and test datasets
- Implement the prediction analisys using Logistic Regression
- Evaluate the model performance and explain the results


## Customer Segmentation: 
- Segment the customers in groups
- Do a behavior analysis  of each group and suggest marketing strategies for each one.



In [ ]:
customer_segmentation = duckdb.query(
"""
  SELECT
    CASE
      WHEN customer_state IN ('DF', 'GO', 'MS') THEN 'Centro-Oeste'
      WHEN customer_state IN ('ES', 'MG', 'RJ', 'SP') THEN 'Sudeste'
      WHEN customer_state IN ('PR', 'RS', 'SC') THEN 'Sul'
      WHEN customer_state IN ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') THEN 'Nordeste'
      WHEN customer_state IN ('AC','AP','AM','PA','RO','RR','TO') THEN 'Norte'
      ELSE 'Other Region'
    END customer_region,

    CASE
        WHEN delivery_distance >= 2500 THEN 'Long Delivery Distance'
        WHEN delivery_distance >= 1000 THEN 'Mid Delivery Distance'
        ELSE 'Short Delivery Distance'
    END AS delivery_distance_group,

    CASE
        WHEN price >= 2500 THEN 'High Medium Ticket'
        WHEN price >= 750 THEN 'Mid Medium Ticket'
        ELSE 'Low Medium Ticket'  
    END AS order_price_group,
    
    COUNT(DISTINCT customer_unique_id) AS count_customer,
    COUNT(DISTINCT order_id) AS total_orders,
    COUNT(order_id) AS total_products,
    ROUND(AVG(price), 2) AS avg_price,
    ROUND(SUM(price), 2) AS total_price,
    ROUND(AVG(freight_value), 2) AS avg_freight_value,
    ROUND(AVG(delivery_distance), 2) AS avg_delivery_distance
  FROM
    master_table
  GROUP BY
    1,2,3
  ORDER BY
    4 DESC
"""
).df()

customer_segmentation

### Marketing Strategies
- Making freight free offers on Short Delivery Distance for Low Medium Ticket in the Sudeste and Sul regions as they are the Top 1 and Top 2 in revenue and product count, as they freight costs are almost 20% of the product cost, on average. This strategy could decresce the customer acquisition prices, building customer loyalty.
- For Norte and Nordeste regions, the majority of the customers are in the Mid to Long Delivery Distances, with high freight costs, the strategy here of having discounts in cumulative products in the same shopping cart.


## Customer Satisfaction: 
- Explore the relationship between customer review score and other aspects, like product category, delivery time, average order price.
- Identify the factors that impact the most

In [ ]:
# review score and delivery distance 

delivery_distance_review = duckdb.query(
"""
    SELECT
        CASE
            WHEN delivery_distance >= 2500 THEN '2500 Km +'
            WHEN delivery_distance >= 1500 THEN '1500 - 2499 Km'
            WHEN delivery_distance >= 1000 THEN '1000 - 1499 Km'
            WHEN delivery_distance >= 750 THEN '750 - 999 Km'
            WHEN delivery_distance >= 500 THEN '500 - 749 Km'
            WHEN delivery_distance >= 250 THEN '250 - 499 Km'
            WHEN delivery_distance >= 100 THEN '100 - 249 Km'
            ELSE 'Up to 50 Km'  
        END AS delivery_distance,
       
        ROUND(AVG(review_score), 2) AS avg_review_score,
        COUNT(order_id) AS total_orders
    FROM
        master_table
    WHERE
        review_score >= 1
    GROUP BY
        1
    ORDER BY
        2
"""
).df()

delivery_distance_review

### Results:
- By looking at delivery distances, we can see that the average after 1000km goes lower than 4.0, meaning that farthest the seller is the lower the review score gets, getting on a stale average between 250 and 999 km and up to 249 the review score average goes to 4.21, the biggest average

In [ ]:
# review score and product category

product_category_review = duckdb.query(
"""
    SELECT
        product_category_name,
       
        ROUND(AVG(review_score), 2) AS avg_review_score,
        COUNT(order_id) AS total_orders
    FROM
        master_table
    WHERE
        review_score >= 1
    GROUP BY
        1
    ORDER BY
        2
"""
).df()

product_category_review

In [ ]:
# review score and delivery time

delivery_time_review = duckdb.query(
"""
    SELECT
        CASE
            WHEN date_diff('day', order_approved_at, order_delivered_customer_date) <= 5 THEN 'Up to 5 Days'
            WHEN date_diff('day', order_approved_at, order_delivered_customer_date) <= 10 THEN 'Up to 10 Days'
            WHEN date_diff('day', order_approved_at, order_delivered_customer_date) <= 15 THEN 'Up to 15 Days'
            WHEN date_diff('day', order_approved_at, order_delivered_customer_date) <= 20 THEN 'Up to 20 Days'
            WHEN date_diff('day', order_approved_at, order_delivered_customer_date) <= 25 THEN 'Up to 25 Days'
            WHEN date_diff('day', order_approved_at, order_delivered_customer_date) <= 30 THEN 'Up to 30 Days'
            ELSE '30+ Days'
        END days_to_delivery,

        ROUND(AVG(review_score), 2) AS avg_review_score,
        COUNT(order_id) AS total_orders
    FROM
        master_table
    WHERE
        review_score >= 1
        AND order_status = 'delivered'
    GROUP BY
        1
    ORDER BY
        2
"""
).df()

delivery_time_review


### Results:
- As we can see, the longer the days to delivered, the lower is the Average Review Score

In [ ]:
# review score and average order price

average_price_review = duckdb.query(
"""
    WITH sum_orders_prices AS (
        SELECT
            order_id,
            customer_unique_id,
            MAX(review_score) AS review_score,
            SUM(price) AS price
        FROM
            master_table
        WHERE
            order_status = 'delivered'
        GROUP BY
            1,2
    )

    SELECT
        CASE
            WHEN price <= 50 THEN 'Up to R$50'
            WHEN price <= 100 THEN 'R$100 - R$149'
            WHEN price <= 150 THEN 'R$150 - R$249'
            WHEN price <= 250 THEN 'R$250 - R$499'
            WHEN price <= 500 THEN '$500 - R$1499'
            ELSE 'R$1500 +'
        END days_to_delivery,

        ROUND(AVG(review_score), 2) AS avg_review_score,
        COUNT(order_id) AS total_orders
    FROM
        sum_orders_prices
    GROUP BY
        1
    ORDER BY
        2
"""
).df()

average_price_review

### Results:
- The Review Averages were pretty balanced, but the higher the prices more criticism towards the orders

### Overall results: 
- The biggest issue found was delivery time, it has the lowest average review scores of all the 4 tests, followed by some Product Categories

# Part 4:

## Data Visualization

- Visualization and Dashboarding
    - MoM Barchart of Orders filtered by Brazil States and Product Category
    - Heatmap of Orders by Region/Brazil States
    - Multiple graphs show the relationship between customer satisfaction and delivery time
    - A dashboard comparing different vendors with best performance in sales, customer satisfaction and delivery time

### Dashboard is acessable in this link: https://lookerstudio.google.com/reporting/4c659dbf-b589-4d1a-8e82-a980ac948a77